In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name



/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [2]:
import os
os.getcwd()

'/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1'

In [5]:
def load_csv_dataset(file_path):
    return pd.read_csv(file_path)


cyrillic_guys = load_csv_dataset('cyrillic_dudes.csv')
lookup_guys = load_csv_dataset('lookup_required_dudes.csv')

online_lookup_1_found = load_csv_dataset('Online lookup required - 1 name found.csv')
online_lookup_0_found = load_csv_dataset('Online lookup required - 0 names found.csv')
online_lookup_multi_found = load_csv_dataset('Online lookup required - multiple names found.csv')

In [8]:
#LOOKUP DATASETS

#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

result_names_null_values = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/nullnames_values.csv')
result_names_null_values = list(result_names_null_values['Name'])


/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_62587/1390145743.py:2: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


# Helper methods

In [10]:
#lookup required - helper functions
from fuzzywuzzy import process

def find_close_matches(this_jersey, dataset_nationality, threshold=90):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]


def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]


def rearrange_name_asian(this_jersey):
    """
    Rearrange the name in `this_jersey` so that the last token comes first followed by a space.

    Args:
    - this_jersey (str): The name to rearrange.

    Returns:
    - Rearranged name string.
    """
    tokens = this_jersey.split()
    rearranged_name = tokens[-1] + " " + " ".join(tokens[:-1])
    return rearranged_name

import unicodedata


def normalize_name(name):
    """
    Normalize the given name by removing dashes, hyphens, apostrophes, and backticks,
    and replacing characters with accents with their unaccented counterparts.

    Args:
    - name (str): The name to normalize.

    Returns:
    - Normalized name string.
    """
    # Remove dashes, hyphens, apostrophes, and backticks
    name = re.sub(r'[-\'`]', '', name)

    # Replace characters with accents with their unaccented counterparts
    name = ''.join(char for char in unicodedata.normalize('NFD', name) if unicodedata.category(char) != 'Mn')

    return name

def switch_yi(string):
    if string.endswith('yi'):
        switched_string = string[:-2] + 'iy'
        return switched_string
    elif string.endswith('iy'):
        switched_string = string[:-2] + 'yi'
        return switched_string
    else:
        return string
    

def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        print("No matches found even with the lowest threshold. jersey was ", this_jersey)
        return 0, 50

def filter_matches_by_initial_and_name_start(this_jersey_tokens, matches):
    """
    Filter matches based on whether at least one token starts with the initial and at least one token starts with 
    the first character of the name, with additional conditions based on the tokens in this_jersey_tokens.

    Args:
    - this_jersey_tokens (list): List containing the initial and name parts of the jersey.
    - matches (list): List of matches to filter.

    Returns:
    - List of filtered matches.
    """
    filtered_matches = []
    
    # Check if this_jersey_tokens has multiple tokens
    if len(this_jersey_tokens) > 1:
        initial_starts_with = this_jersey_tokens[0]
        name_starts_with = this_jersey_tokens[1][0]

        # Additional logic for handling 'H' and 'Kh'
        if name_starts_with == 'H':
            if any(token.startswith('Kh') for token in this_jersey_tokens[1:]):
                name_starts_with = 'Kh'

        # Additional logic for handling multiple tokens
        for token in this_jersey_tokens[1:]:
            if token[0] != name_starts_with:
                initial_starts_with = None
                break
        
        for match in matches:
            match_tokens = match.split()
            initial_found = False
            name_start_found = False

            # Check if at least one token starts with the initial
            if initial_starts_with:
                for token in match_tokens:
                    if token.startswith(initial_starts_with):
                        initial_found = True
                        break

            # Check if at least one token starts with the first character of the name
            for token in match_tokens:
                if token.startswith(name_starts_with) or (name_starts_with == 'H' and token.startswith('Kh')):
                    name_start_found = True
                    break

            # Check if any token starts with any character of the hyphenated name
            if '-' in this_jersey_tokens[1]:
                hyphenated_parts = this_jersey_tokens[1].split('-')
                for part in hyphenated_parts:
                    for token in match_tokens:
                        if token.startswith(part[0]):
                            name_start_found = True
                            break

            # If both conditions are met, add the match to filtered_matches
            if initial_found and name_start_found:
                filtered_matches.append(match)

    else:  # Handle the case where this_jersey_tokens has only one token
        for match in matches:
            match_tokens = match.split()
            initial_found = False
            name_start_found = False

            # Check if at least one token starts with the initial
            for token in match_tokens:
                if token.startswith(this_jersey_tokens[0]):
                    initial_found = True
                    break

            # Check if at least one token starts with the first character of the name
            for token in match_tokens:
                if token.startswith(this_jersey_tokens[0][0]):
                    name_start_found = True
                    break

            # Check if any token starts with any character of the hyphenated name
            if '-' in this_jersey_tokens[0]:
                hyphenated_parts = this_jersey_tokens[0].split('-')
                for part in hyphenated_parts:
                    for token in match_tokens:
                        if token.startswith(part[0]):
                            name_start_found = True
                            break

            # If both conditions are met, add the match to filtered_matches
            if initial_found and name_start_found:
                filtered_matches.append(match)

    return filtered_matches

# this_jersey_tokens = ['R', 'Hojayev']
# matches = ['Resul Khozhaev']
# print(filter_matches_by_initial_and_name_start_new(this_jersey_tokens, matches))

def check_last_token_match(this_jersey, matches):
    """
    Check if the last token of each match matches the jersey with variations like accents or substitutions.

    Args:
    - this_jersey (str): The original name being searched for.
    - matches (list): Set of names matching the jersey.

    Returns:
    - List of matches where the last token matches the jersey with variations.
    """
    def remove_accents(input_str):
        """
        Remove accents from characters in a string.
        """
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    def apply_substitutions(input_str):
        """
        Apply specific character substitutions in a string.
        """
        substitutions = {'g': 'h', 'ij': 'y', 'zh': 'j', 'ae': 'aye', 'H': 'Kh'}
        for key, value in substitutions.items():
            input_str = input_str.replace(key, value)
        return input_str

    jersey_last_token = this_jersey.split()[-1]

    matching_matches = []
    for match in matches:
        match_last_token = match.split()[-1]

        # Check if the last token matches the jersey (ignoring accents)
        if remove_accents(match_last_token) == remove_accents(jersey_last_token):
            matching_matches.append(match)
        else:
            # Check if the last token matches the jersey with specific character substitutions
            if apply_substitutions(match_last_token) == apply_substitutions(jersey_last_token):
                matching_matches.append(match)

    return matching_matches

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches




# Cyrillic

# Lookup Required

In [6]:
lookup_guys

,Unnamed: 0.1,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Status II,Status III
0,1148,48,2681,"World Cup Qualifiers, Asia, 2022",Thailand vs Indonesia,03-06-2021,Thailand,TH,2021,FAIL,0,0.0,True,Lookup Required,False,0,E Amantegui,none,NaN,NaN
1,1164,64,2681,"World Cup Qualifiers, Asia, 2022",Thailand vs Indonesia,03-06-2021,Indonesia,ID,2021,FAIL,0,0.0,True,Lookup Required,False,0,W Sulaeman,none,NaN,NaN
2,1200,100,2683,"World Cup Qualifiers, Asia, 2022",Saudi Arabia vs Yemen,05-06-2021,Yemen,YE,2021,FAIL,0,0.0,True,Lookup Required,False,0,Ala Addin Mahdi,none,NaN,NaN
3,1259,159,2686,"World Cup Qualifiers, Asia, 2022",Turkmenistan vs North Korea,14-11-2019,Turkmenistan,TM,2019,FAIL,0,0.0,True,Lookup Required,False,0,W Ylýasow,none,NaN,NaN
4,1271,171,2686,"World Cup Qualifiers, Asia, 2022",Turkmenistan vs North Korea,14-11-2019,North Korea,KP,2019,FAIL,0,0.0,True,Lookup Required,False,0,Y Lee,none,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,96402,636,1495,Nations League 2018-2020,Latvia vs Georgia,16.10.18,Georgia,GE,2018,FAIL,0,0.0,True,Lookup Required,False,0,Gviliya,none,NaN,NaN
2022,96445,679,1497,Nations League 2018-2020,Andorra vs Georgia,15.11.18,Georgia,GE,2018,FAIL,0,0.0,True,Lookup Required,False,0,Gviliya,none,NaN,NaN
2023,96479,713,1499,Nations League 2018-2020,Georgia vs Kazakhstan,19.11.18,Georgia,GE,2018,FAIL,0,0.0,True,Lookup Required,False,0,Gviliya,none,NaN,NaN
2024,96585,819,1504,Nations League 2018-2020,Belarus vs Luxembourg,12.10.18,Belarus,BY,2018,FAIL,0,0.0,True,Lookup Required,False,0,Putsila,none,NaN,NaN


# Online Lookup Required

In [11]:
dataframe_matches_2 = pd.DataFrame()
for index, row in online_lookup_1_found.iterrows():
    this_row_success_tally = 0
    this_row_multi_tally = 0
    this_row_fail_tally = 0
# for i in range(0, 1):
#     index = 2348
#     row = combined_df_failed_2.iloc[index]

    name_list_matches = []

    this_jersey = row['ORIGINAL JERSEY']
    
    nationality = row['Nationality']
    nationality_code = row['Team Country Code']
    if nationality_code == 'BVI':
        nationality_code = ' BVI'
    country_name = countries_codes[countries_codes[' Code'] == nationality_code]['Country'].unique()[0]
    if nationality_code == ' DR':
        nationality_code = 'DR'
    season = row['Season']
    match_date = row['Date']
    dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == nationality_code]['Name'].unique()

    if this_jersey.endswith('yi'):
        this_jersey = switch_yi(this_jersey)
    elif this_jersey.endswith('iy'):
        this_jersey = switch_yi(this_jersey)

    name_type = ''
    matches, threshold_num = threshold_player_match(this_jersey, dataset_nationality)

    this_jersey_tokens = this_jersey.split(' ')

Player is Y sako. Found matches: ['Yuya Osako']. threshold is 86
Player is Qu Ngc Hi. Found matches: ['Ngoc Hai Que']. threshold is 82
Player is Phm Tun Hi. Found matches: ['Tuan Hai Pham']. threshold is 83
Player is Mohsin Al Khaldi. Found matches: ['Mohamed Saleh Al-Ghassani', 'Mohammed Alo Humaid Al Seyabi', 'Abdulrahman Al-Mushaifri']. threshold is 86
Player is Ouk Sovann. Found matches: ['Sovann Ouk']. threshold is 89
Player is Ahmad Ibrahim Al-Qafaje. Found matches: ['Karrar Ibrahim', 'Ahmad Mnajed', 'Kais Al-Ani', 'Ibrahim Bayesh', 'Amir Al-Ammari', 'Ahmad Fadhil', 'Ali Al-Hamadi', 'Ahmad Allée', 'Louaï Al-Ani', 'André Al-Sanati', 'Danilo Al-Saed']. threshold is 86
Player is Mohsin Al Khaldi. Found matches: ['Mohamed Saleh Al-Ghassani', 'Mohammed Alo Humaid Al Seyabi', 'Abdulrahman Al-Mushaifri']. threshold is 86
Player is Qu Ngc Hi. Found matches: ['Ngoc Hai Que']. threshold is 82
Player is Nguyn Hoàng c. Found matches: ['Phong Hong Duy Nguyen']. threshold is 86
Player is Fahad

In [9]:
#online lookup - 1 guy found 
online_lookup_1_found

#if the name is initial + name make sure the name starts with initial and ends with name. be mindful of the accents here if names are same w accents removed that's ok

#if its just name (no initials) do nothing 

#is the name that it found the closest match in the dataset_nationality?
#is there another name in the dataset thats basically a better quality of match?? or the same? 
    #if there is another potential name do you do the multi name match date lookup? 

        #if one guy is remaining YOU HAVE A MATCH

        #if multiple remaining, then see the below example when there's an array of names

        #if 0 remaining then what?

    #if not any other potential names, evaluate the quality of this match. 

        #if the quality is good then YOU HAVE A MATCH

        #if the quality is super super low then what?  

    #provided you have a name 
    #look up that name + that season 
    #if there are no rows, you have to look them up online 

,Unnamed: 0.1,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
0,1176,76,2682,"World Cup Qualifiers, Asia, 2022",Japan vs Saudi Arabia,01-02-2022,Japan,JP,2022,FAIL,Yuya Osako,0.0,True,Online Lookup Required,False,Yuya Osako,Y sako,single
1,1290,190,2687,"World Cup Qualifiers, Asia, 2022",Japan vs Vietnam,29-03-2022,Vietnam,VN,2022,FAIL,Thanh Hien Nguyen,0.0,True,Online Lookup Required,False,Thanh Hien Nguyen,Qu Ngc Hi,single
2,1296,196,2687,"World Cup Qualifiers, Asia, 2022",Japan vs Vietnam,29-03-2022,Vietnam,VN,2022,FAIL,Minh Tuan Ha,0.0,True,Online Lookup Required,False,Minh Tuan Ha,Phm Tun Hi,single
3,1329,229,2689,"World Cup Qualifiers, Asia, 2022",Oman vs India,19-11-2019,Oman,OM,2019,FAIL,Mohsin Al Harbi,0.0,True,Online Lookup Required,False,Mohsin Al Harbi,Mohsin Al Khaldi,single
4,1366,266,2691,"World Cup Qualifiers, Asia, 2022",Cambodia vs Hong Kong,05-09-2019,Cambodia,KH,2019,FAIL,Ouk Sothy,0.0,True,Online Lookup Required,False,Ouk Sothy,Ouk Sovann,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,95459,793,3270,"World Cup Qualifiers, Europe, 2022",Iceland vs Germany,08-09-2021,Iceland,IS,2021,FAIL,Bödvar Bödvarsson,0.0,True,Online Lookup Required,False,Bödvar Bödvarsson,B Sævarsson,single
1254,95467,801,3270,"World Cup Qualifiers, Europe, 2022",Iceland vs Germany,08-09-2021,Iceland,IS,2021,FAIL,Helgi Danielsson,0.0,True,Online Lookup Required,False,Helgi Danielsson,Þ Helgason,single
1255,95701,1035,3281,"World Cup Qualifiers, Europe, 2022",Iceland vs North Macedonia,05-09-2021,Iceland,IS,2021,FAIL,Bödvar Bödvarsson,0.0,True,Online Lookup Required,False,Bödvar Bödvarsson,B Sævarsson,single
1256,95967,201,1476,Nations League 2018-2020,Lithuania vs Serbia,07.09.18,Lithuania,LT,2018,FAIL,Arunas Klimavicius,0.0,True,Online Lookup Required,False,Arunas Klimavicius,Ar Klimavičius,single


In [ ]:
#online lookup required - multiple names found 


#if multiple names were found

    #if the Jersey has an initial 
        #keep all matches that start with the initial 

        #except if the initial is a lowercase d. then, make sure there is a word starting with d followed by the Name 
        #so if its d Silva make sure theres a word startign with d then Silva soemwhere in the name. 

    #sadiqov case - what if the names are the same???
        #in this case you should just look up each name and use like whichever has the record from the saeason of the game 

    #do the match date lookup thing with each of the matches to check who played in that game. 
        #if you still have more than 1 name

            #look at the success dataset to find the record of each remaining guy
                #if its ivanovic and you find both B and D Ivanovic played according to transfermarkt
                
                #check if there's a record of those guys in that match / date (in our dataset)
                    
                    #if all but one guy is in the success dataset, the guy who is not there is the name you need to use.

                    #if none remain after checking the dataset just remove this row. but print that out. that would be weird

                    #if multiple remain after checking the success dataset(there were 3 names remaining, only found 1 record)
                        #print this out this is a special case.

        #if you are just left with one name using the match date lookup 

            #that's a success - find that guy in the database